In [93]:
import PyPDF2
import re
import pandas as pd
import numpy as np

In [94]:

def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

## Book 1 : Gramer Kreol Morisien (Police-Michel, Carpooran & Florigny)

In [95]:
pdf_path = r"C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\books\Mauritian Creole; Gramer Kreol Morisien (Police-Michel, Carpooran & Florigny).pdf"
extracted_text = extract_text_from_pdf(pdf_path)

In [96]:
pdf_reader = PyPDF2.PdfReader(pdf_path)

In [151]:
list_of_en_cr_pages = [
    (5,6),
]
list_of_cr_fr_pages = [
    (7,8)
]
pages_to_manually_parse =  list(range(40,46)) + list(range(51,73)) + list(range(75,91)) + list(range(92,115)) + [118 , 119] + list(range(123,149))
omit_pages = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18] + pages_to_manually_parse + list(range(150,172))
list_of_cr_pages = [x for x in range(len(pdf_reader.pages)) if x not in omit_pages]

In [98]:
def remove_num_npage_space(str):
    cleaned_string = re.sub(r'[\s\d]+', ' ', str).strip()
    return cleaned_string

### fr_cr manual parsing 

In [99]:

pg7 = pdf_reader.pages[7].extract_text()
pg8 = pdf_reader.pages[8].extract_text()

pg7 = remove_num_npage_space(pg7)
pg8 = remove_num_npage_space(pg8)

In [107]:
pg7_split = pg7.split('.')
pg7_split.pop(1)
pg8_split = pg8.split('.')


' Sa zour-la, lang maternel nou pei finn ofisielman rant dan lekol'

In [119]:
raw_fr_cr_df = pd.DataFrame()

In [219]:
def add_lists_to_dataframe(df, list1, list2, col1,col2):
    diff = len(list1) - len(list2)
    if diff > 0:
        list2 += [''] * diff
    elif diff < 0:
        list1 += [''] * abs(diff)
    df[col1] = list1
    df[col2] = list2
    
    return df


In [121]:
cr_fr_parallel_raw = add_lists_to_dataframe(raw_fr_cr_df,pg8_split,pg7_split,'fr','cr') #removed last 3 rows by hand. just names

In [124]:
cr_fr_parallel_raw.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\fr_cr_parallel_raw.csv',index=False)

### en_cr manual translation

In [138]:
pg5 = pdf_reader.pages[5].extract_text()
pg6 = pdf_reader.pages[6].extract_text()
pg5 = remove_num_npage_space(pg5)
pg6 = remove_num_npage_space(pg6)
pg5_split = pg5.split('.')
pg6_split = pg6.split('.')

In [139]:
raw_en_cr_df = pd.DataFrame()

In [145]:
raw_en_cr_parallel = add_lists_to_dataframe(raw_en_cr_df,pg5_split,pg6_split,'en','cr',3)

In [146]:
raw_en_cr_parallel.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\en_cr_parallel_raw.csv',index=False)

### monolingual creole text parsing

In [185]:
import unicodedata

In [187]:
def remove_special_characters(raw_str):
    return ''.join(char for char in raw_str if unicodedata.category(char)[0] != 'C')

In [195]:
cr_sentences = []
for page in list_of_cr_pages:
    raw_str = remove_num_npage_space(pdf_reader.pages[page].extract_text())
    raw_str = remove_special_characters(raw_str)
    raw_str_split = raw_str.split('.')
    raw_str_split = [x.strip(' ') for x in raw_str_split if len(x)>4]
    cr_sentences.extend(raw_str_split)

In [197]:
raw_cr_df = pd.DataFrame(cr_sentences,index=range(len(cr_sentences)),columns=['cr'])

In [199]:
raw_cr_df.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_monolingual_raw.csv',index=False)

### ToDo: pages to manually parse

## Book 2 : Lortograf Kreol Morisien (Carpooran)

In [253]:
list_of_en_cr_pages = [
    (5,6),(8,9)
]
list_of_cr_fr_pages = [
    (8,10)
]
omit_pages = [12,21,22,23,24,25,26,36,37,38,39] + list(range(54,70)) + list(range(0,12))
list_of_cr_pages = [x for x in range(len(pdf_reader.pages)) if x not in omit_pages]

In [202]:
pdf_path = r"C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\books\Lortograf Kreol Morisien.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
pdf_reader = PyPDF2.PdfReader(pdf_path)

In [212]:
def make_parallel(pdfObj,pg1,pg2,src_lang,tgt_lang):
    pg5 = pdfObj.pages[pg1].extract_text()
    pg6 = pdfObj.pages[pg2].extract_text()
    pg5 = remove_num_npage_space(pg5)
    pg6 = remove_num_npage_space(pg6)
    pg5 = remove_special_characters(pg5)
    pg6 = remove_special_characters(pg6)
    pg5_split = pg5.split('.')
    pg6_split = pg6.split('.')
    if len(pg5_split) != len(pg6_split):
        print(f"{src_lang} & {tgt_lang} text is not parallel.")
    return pg5_split,pg6_split

In [213]:
pg5_split,pg6_split = make_parallel(pdf_reader,5,6,'en','cr')

en & cr text is not parallel.


In [220]:
raw_cr_en_temp_df = pd.DataFrame()
raw_cr_en_temp_df_parallel_raw = add_lists_to_dataframe(raw_cr_en_temp_df,pg5_split,pg6_split,'en','cr') 

In [222]:
raw_cr_en_temp_df_parallel_raw.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_en_parallel_temp_raw.csv',index=False)

In [227]:
pg9_split,pg8_split = make_parallel(pdf_reader,9,8,'en','cr')

en & cr text is not parallel.


In [232]:
raw_cr_en_temp_df2 = pd.DataFrame()
raw_cr_en_temp_df_parallel_raw2 = add_lists_to_dataframe(raw_cr_en_temp_df2,pg9_split,pg8_split,'en','cr')

In [234]:
raw_cr_en_temp_df_parallel_raw2.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_en_parallel_temp_raw2.csv',index=False)

made 2 parallel cr->en files which are temp. will join all temp files later

In [236]:
pg8_split,pg10_split = make_parallel(pdf_reader,8,10,'cr','fr')

In [245]:
raw_cr_fr_temp_df = pd.DataFrame()
raw_cr_fr_temp_df_parallel_raw = add_lists_to_dataframe(raw_cr_fr_temp_df,pg10_split,pg8_split,'fr','cr') 

In [248]:
raw_cr_fr_temp_df_parallel_raw.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_fr_parallel_temp_raw.csv',index=False)

### cr monolingual 

In [257]:
cr_sentences = []
for page in list_of_cr_pages:
    raw_str = remove_num_npage_space(pdf_reader.pages[page].extract_text())
    raw_str = remove_special_characters(raw_str)
    raw_str_split = raw_str.split('.')
    raw_str_split = [x.strip(' ') for x in raw_str_split if len(x.split(' '))>4]
    cr_sentences.extend(raw_str_split)

In [259]:
len(cr_sentences)

426

In [261]:
raw_cr_df = pd.DataFrame(cr_sentences,index=range(len(cr_sentences)),columns=['cr'])

In [265]:
raw_existing_cr = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_monolingual_raw.csv')

In [266]:
raw_existing_cr = pd.concat([raw_existing_cr,raw_cr_df])

In [268]:
raw_existing_cr.reset_index(inplace=True,drop=True)

In [269]:
raw_existing_cr.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_monolingual_raw.csv',index=False)

### merging existing temp

In [272]:
save_path = r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw'

In [273]:
base = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\en_cr_parallel_raw.csv')
temp1 = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_en_parallel_temp_raw.csv')
temp2 = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_en_parallel_temp_raw2.csv')

In [277]:
merged = pd.concat([base,temp1,temp2])

In [279]:
merged.reset_index(inplace=True,drop=True)

In [282]:
merged.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\en_cr_parallel_raw.csv')

In [283]:
base = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\fr_cr_parallel_raw.csv')
temp1 = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\cr_fr_parallel_temp_raw.csv')

In [284]:
merged = pd.concat([base,temp1])

In [286]:
merged.reset_index(inplace=True,drop=True)

In [287]:
merged.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\fr_cr_parallel_raw.csv',index=False)

## Book 3: A reflection on scientific and mathematical terms in Mauritian Kreol at primary level: an overview
paper has 1-1 translations of scientific terms

In [288]:
pages_to_parse = list(range(14,18)) + [20]

In [289]:
pdf_path = r"C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\books\A_reflection_on_scientific_and_mathematical_terms_.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
pdf_reader = PyPDF2.PdfReader(pdf_path)

In [297]:
for i in pages_to_parse:
    print(pdf_reader.pages[i].extract_text())

14 
  
 
Mathematical terms and phrases  
Word categories for mathematics  
Green words : High T/C vehicularity [equivalents already exist in the dictionary/common use 
of language to denote same or similar reality]  
 
Word/phrase in the source language  Word/phrase in the target language  
Addition  Adision  
Acute angle  Lang egi  
Anticlockwise  Kont sans enn mont  
Arc Ark 
Ascending o rder Lord montan/  
Average  Mwayenn  
Calculate  Kalkile  
Cardinal number  Nonb kardinal  
Clockwise  Dan sans enn mont  
Coordinates  kordone  
Cylinder  Silenn  
Decimal  Desimal  
Digit  sif 
Find the difference  Rod diferans  
If-then statement  Si-alor 
Magnitude  Magn itid 
Mean  Mwayenn  
Minus  Retire  
Multiple  miltip  
Ordinal numbers  Nonb ordinal  
Parabola  Parabol  
Pentagon, hexagon, heptagon,...decagon  Pentagonn, hexagonn, heptagonn, ...  
Perpendicular  Perpandikiler  
Plus Plis 
Polygon  Poligonn  
Product  Prodwi  
Quadrant  Kadran  
Ratio  Proporsion  
Rational  Rasionel  
So

In [295]:
'''
Addition  Adision  
Acute angle  Lang egi  
Anticlockwise  Kont sans enn mont  
Arc Ark 
Ascending o rder Lord montan/  
Average  Mwayenn  
Calculate  Kalkile  
Cardinal number  Nonb kardinal  
Clockwise  Dan sans enn mont  
Coordinates  kordone  
Cylinder  Silenn  
Decimal  Desimal  
Digit  sif 
Find the difference  Rod diferans  
If-then statement  Si-alor 
Magnitude  Magn itid 
Mean  Mwayenn  
Minus  Retire  
Multiple  miltip  
Ordinal numbers  Nonb ordinal  
Parabola  Parabol  
Pentagon, hexagon, heptagon,...decagon  Pentagonn, hexagonn, heptagonn, ...  
Perpendicular  Perpandikiler  
Plus Plis 
Polygon  Poligonn  
Product  Prodwi  
Quadrant  Kadran  
Ratio  Proporsion  
Rational  Rasionel  
Solve  Rezoud  
Take away  Pran/ tir  
Trapezium  Trapez  
What is the sum  Ki som  
What is the total  Ki total.. 

Adjacent  Adjacent  
Apex  Apex  
Base (geometry)  Base  
Cross section  Cross -section [lakoup transversal??]  
Highest common factor  Highest common factor  
Kite Kite 
Least common multiple  Least common multiple  
Subset  Subset  
Matrix  Matrix  
 
Blue words:  Intermediate vehicularity [use of both forms {original loanword or new 
translated form} can be possible]  
Word/phrase in the source language  Word/phrase in the target language  
Axis (graph)  Axis/ Lax 
Curve  Kourb  
Descending order  Descending order  
Empty set  Set vid  
Equilateral triangle  Triyang ekwilateral  
Factor  Fakter  
Median  Mediann  
Mode  Mod (/mo:d/)  
Right angle  Lang drwat  
Right cylinder  Silenn drwat  
Sector of a circle  Sekter enn serk  
Segment of a circle  Segman enn serk  
Set Union  Linion  
 
Word categories for science  
Green words : High T/C vehicularity [equivalents already exist in the dictionary/common use 
of language to denote s ame or similar reality]  
Word/phrase in the source language  Word/phrase in the target language  
Argon, boron, krypton  Argonn, Boronn, Kryptonn  
As shown in  Kouma finn montre anba/dan  
Atmosphere  latmosfer  
Atom  Atom  
Bacteria  Bakteri  
Barometric pressure  Presion barometrk 

Biomass  Biomas  
Canned foods  Aliman/manze dan bwat/konserv ou konserv  
Carcinogen  kanserizenn  
Clay soil  (later) arzil  
Contaminant  Kontaminan  
Crawl  Ranpe  
Density  Dansite  
Diagram  Diagram  
Disintegrate  Dezintegre  
Displacement  Deplasman / Displacement  
Dissolve  Disoud/ fonn  
Distance  Distans  
Estuary  Estier  
Fauna  Lafonn  
Filtered and purified  Filtre ek pirifie  
Fin Nazwar  
Flora  Laflor  
Form  Form  
Fossil fuel  Lenerzi fosil/ karbiran fosil  
Germs  Mikrob  
Give one other way to kill the ger ms in 
filtered water  Donn enn lot fason pou touy mikrob dan delo 
filtre 
Habitat  Abita  
Hazardous waste  Dese toxik  
How each child is caring for his/her pet 
animal  Kouma sak zanfan pe pran so zanimo swin  
I prepare two potted plants of the same type  Mo pre par/pran de plant dan po  
Insects  Insek  
It is a bad habit to talk while eating  Li pa enn bon labitid koze labous plin  
Matter  Matier  
Microbes  Mikrob  
Movement  Mouvman  
Nitrogen  Nitrozenn  
Permeability  Permeabilite  
Phase (moon)  Faz 
Porosity  Porozite  
Quantity  Kantite  
Saturation  Satirasion  
Sense organs  Organn sans  
Speed  Vites  
Sunlight  Lalimier soley  
Surface water  Delo sirfas  
Tail Lake  
Thermal power station  Stasion termik  
Time  Letan  
Twigs  Ti baton  
Velocity  Velosite  
Volume  Volim  
Waste  Dese  
 

Inorganic compounds  Konpoze inorganik  
Mass  Pwa  
My ears are my sense of hearing  Mo tande avek mo zoreye  
My hands are my sense of touch  Mo touse avek mo lame  
Scale  Lesel  
Sight, hearing, smell, taste and touch  Lavi, lwi, lodora, gou, touse  
To scale  Eselone  
Wind machines  Masinn divan  

Scale (noun)  Creole equivalent  
(scale classification)  Barem  
(scale as magnitude) Inportans  
(fish scale)  Lekay  
(scale for measure)  Gradiasion  
(scale to measure mass)  Balans  
Scale  Lesel  
 
Scale (verb)  Creole equivalent  
To measure  Mezire  
To weigh  Peze  
To scale (fish)  Tir lekay  
To scale (Reduce/enlarge)  Diminie/agrandi  
'''

['__abstractmethods__',
 '__annotations__',
 '__callable_proto_members_only__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parameters__',
 '__protocol_attrs__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_transformation_matrix',
 '_clone',
 '_content_stream_rename',
 '_debug_for_extract',
 '_expand_mediabox',
 '_extract_text',
 '_get_fonts',
 '_is_protocol',
 '_is_runtime_protocol',
 '_merge_page',
 '_merge_resources',
 '_push_pop_gs',
 '_reference_clone',
 'addTransformation',
 'add_transformation',
 'annotations',
 'artBox',
 'artbox',
 'bleedBox',
 'bleedbo

In [299]:
data = {
    'en': ['Addition', 'Acute angle', 'Anticlockwise', 'Arc', 'Ascending order', 'Average', 'Calculate', 'Cardinal number', 'Clockwise', 'Coordinates', 'Cylinder', 'Decimal', 'Digit', 'Find the difference', 'If-then statement', 'Magnitude', 'Mean', 'Minus', 'Multiple', 'Ordinal numbers', 'Parabola', 'Pentagon, hexagon, heptagon,...decagon', 'Perpendicular', 'Plus', 'Polygon', 'Product', 'Quadrant', 'Ratio', 'Rational', 'Solve', 'Take away', 'Trapezium', 'What is the sum', 'What is the total'],
    'cr': ['Adision', 'Lang egi', 'Kont sans enn mont', 'Ark', 'Lord montan', 'Mwayenn', 'Kalkile', 'Nonb kardinal', 'Dan sans enn mont', 'kordone', 'Silenn', 'Desimal', 'sif', 'Rod diferans', 'Si-alor', 'Magn itid', 'Mwayenn', 'Retire', 'miltip', 'Nonb ordinal', 'Parabol', 'Pentagonn, hexagonn, heptagonn,...', 'Perpandikiler', 'Plis', 'Poligonn', 'Prodwi', 'Kadran', 'Proporsion', 'Rasionel', 'Rezoud', 'Pran/ tir', 'Trapez', 'Ki som', 'Ki total']
}

data2 = {
    'en': ['Adjacent', 'Apex', 'Base (geometry)', 'Cross section', 'Highest common factor', 'Kite', 'Least common multiple', 'Subset', 'Matrix', 'Axis (graph)', 'Curve', 'Descending order', 'Empty set', 'Equilateral triangle', 'Factor', 'Median', 'Mode', 'Right angle', 'Right cylinder', 'Sector of a circle', 'Segment of a circle', 'Set Union'],
    'cr': ['Adjacent', 'Apex', 'Base', 'Cross-section', 'Highest common factor', 'Kite', 'Least common multiple', 'Subset', 'Matrix', 'Axis/ Lax', 'Kourb', 'Descending order', 'Set vid', 'Triyang ekwilateral', 'Fakter', 'Mediann', 'Mod', 'Lang drwat', 'Silenn drwat', 'Sekter enn serk', 'Segman enn serk', 'Linion']
}

data3 = {
    'en': ['Argon, boron, krypton', 'As shown in', 'Atmosphere', 'Atom', 'Bacteria', 'Barometric pressure', 'Biomass', 'Canned foods', 'Carcinogen', 'Clay soil', 'Contaminant', 'Crawl', 'Density', 'Diagram', 'Disintegrate', 'Displacement', 'Dissolve', 'Distance', 'Estuary', 'Fauna', 'Filtered and purified', 'Fin', 'Flora', 'Form', 'Fossil fuel', 'Germs', 'Give one other way to kill the germs in filtered water', 'Habitat', 'Hazardous waste', 'How each child is caring for his/her pet animal', 'I prepare two potted plants of the same type', 'Insects', 'It is a bad habit to talk while eating', 'Matter', 'Microbes', 'Movement', 'Nitrogen', 'Permeability', 'Phase (moon)', 'Porosity', 'Quantity', 'Saturation', 'Sense organs', 'Speed', 'Sunlight', 'Surface water', 'Tail Lake', 'Thermal power station', 'Time', 'Twigs', 'Velocity', 'Volume', 'Waste'],
    'cr': ['Argonn, Boronn, Kryptonn', 'Kouma finn montre anba/dan', 'Latmosfer', 'Atom', 'Bakteri', 'Presion barometrik', 'Biomas', 'Aliman/manze dan bwat/konserv ou konserv', 'Kanserizenn', '(later) arzil', 'Kontaminan', 'Ranpe', 'Dansite', 'Diagram', 'Dezintegre', 'Deplasman / Displacement', 'Disoud/ fonn', 'Distans', 'Estier', 'Lafonn', 'Filtre ek pirifie', 'Nazwar', 'Laflor', 'Form', 'Lenerzi fosil/ karbiran fosil', 'Mikrob', 'Donn enn lot fason pou touy mikrob dan delo filtre', 'Abita', 'Dese toxik', 'Kouma sak zanfan pe pran so zanimo swin', 'Mo pre par/pran de plant dan po', 'Insek', 'Li pa enn bon labitid koze labous plin', 'Matier', 'Mikrob', 'Mouvman', 'Nitrozenn', 'Permeabilite', 'Faz', 'Porozite', 'Kantite', 'Satirasion', 'Organn sans', 'Vites', 'Lalimier soley', 'Delo sirfas', 'Tail Lake', 'Stasion termik', 'Letan', 'Ti baton', 'Velosite', 'Volim', 'Dese']
}

data4 = {
    'en': ['Inorganic compounds', 'Mass', 'My ears are my sense of hearing', 'My hands are my sense of touch', 'Scale', 'Sight, hearing, smell, taste and touch', 'To scale', 'Wind machines', 'Scale (noun)', '(scale classification)', '(scale as magnitude)', '(fish scale)', '(scale for measure)', '(scale to measure mass)', 'Scale'],
    'cr': ['Konpoze inorganik', 'Pwa', 'Mo tande avek mo zoreye', 'Mo touse avek mo lame', 'Lesel', 'Lavi, lwi, lodora, gou, touse', 'Eselone', 'Masinn divan', 'Lesel', 'Barem', 'Inportans', 'Lekay', 'Gradiasion', 'Balans', 'Lesel']
}

In [309]:
merged_dict = {}

In [310]:
for d in [data, data2, data3, data4]:
    for key, value in d.items():
        if key not in merged_dict:
            merged_dict[key] = value
        else:
            if isinstance(value, list):
                merged_dict[key].extend(value)
            else:
                merged_dict[key] = [merged_dict[key], value]

In [316]:
en_cr_math_words = pd.DataFrame(merged_dict)

In [320]:
existing_df_en_cr = pd.read_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\en_cr_parallel_raw.csv')

In [322]:
existing_df_en_cr.drop(columns={'Unnamed: 0'},inplace=True)

In [324]:
existing_df_en_cr= pd.concat([existing_df_en_cr,en_cr_math_words])

In [325]:
existing_df_en_cr

,en,cr
0,Message from the Minister of Education and Hum...,Mesaz Minis Ledikasion ek Resours Imin Mo bien...
1,This publication on the grammar of Kreol Moris...,"Sa piblikasion lor gramer Kreol Morisien la, ..."
2,We had already gained massive unanimity on the...,Nou finn deza gagn linanimite pou lortograf l...
3,Many amongst us found how useful using both la...,Bokou parmi nou finn fer lexperyans litilite ...
4,Giving the language its credentials has not al...,Donn enn lang so bann referans pa finn touzou...
...,...,...
119,(scale as magnitude),Inportans
120,(fish scale),Lekay
121,(scale for measure),Gradiasion
122,(scale to measure mass),Balans


In [326]:
existing_df_en_cr.to_csv(r'C:\Users\yush\OneDrive\Desktop\papers\kreol-benchmark\data_collection\data_extracted\raw\en_cr_parallel_raw.csv',index=False)

20/02/24 - Parsing books I found online